# Data Wrangling

Now that the data has been collected and is in a clean and tidy format, it's time to make it usable for the question we are trying to answer.

In [1]:
import pandas as pd

df = pd.read_csv('enrollment_data.csv')
df

,time,enrolled,available,waitlisted,total,course,quarter
0,2022-05-18 00:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
1,2022-05-18 12:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
2,2022-05-19 00:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
3,2022-05-19 12:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
4,2022-05-20 00:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
...,...,...,...,...,...,...,...
3070123,2025-01-25 00:00:00,8.0,7.0,0.0,15.0,WES 269,WI 25
3070124,2025-01-25 12:00:00,8.0,7.0,0.0,15.0,WES 269,WI 25
3070125,2025-01-26 00:00:00,8.0,7.0,0.0,15.0,WES 269,WI 25
3070126,2025-01-31 00:00:00,8.0,7.0,0.0,15.0,WES 269,WI 25


Let's first make sure that the time column is consistently formatted.

In [2]:
df['time'] = pd.to_datetime(df['time'])

Then, in order to extract further information, we need to know when registration opens for each quarter.

In order to do this, we will scrape UCSD's publically available yearly "enrollment and registration calendars" to determine the first day of enrollment for each quarter.

In [3]:
import requests
from bs4 import BeautifulSoup

calendar_links = [
    'https://blink.ucsd.edu/instructors/courses/enrollment/calendars/2022.html',    # 2022 - 2023
    'https://blink.ucsd.edu/instructors/courses/enrollment/calendars/2023.html',    # 2023 - 2024
    'https://blink.ucsd.edu/instructors/courses/enrollment/calendars/2024.html'     # 2024 - 2025
]

def process_calendar(link, yr):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    # The website has a table with every important date
    table = soup.find('table')

    # Store enrollment start dates w/ same formatting as the 'quarter' column of the df
    dates = {}

    # Iterate through table rows
    for row in table.find_all('tr'): 
        cells = row.find_all('td')
        if cells:
            label = cells[0].get_text(strip=True)
            if 'Enrollment begins' in label:
                dates['FA' + " " + str(yr)] = cells[1].get_text(strip=True) + "/" + str(yr)

                s = str(yr + 1)
                dates['WI' + " " + s] = cells[2].get_text(strip=True) + "/" + s

                if yr < 24: 
                    dates['SP' + " " + s] = cells[3].get_text(strip=True) + "/" + s

                if yr == 23:
                    dates['S1' + " " + s] = cells[4].get_text(strip=True) + "/" + s
                    dates['S2' + " " + s] = cells[4].get_text(strip=True) + "/" + s
                    dates['S3' + " " + s] = cells[4].get_text(strip=True) + "/" + s
                break
    return dates

enrollment_starts = {}
year = 22
for link in calendar_links:
    enrollment_starts.update(process_calendar(link, year))
    year += 1
enrollment_starts

{'FA 22': '5/20/22',
 'WI 23': '11/7/23',
 'SP 23': '2/18/23',
 'FA 23': '5/26/23',
 'WI 24': '11/14/24',
 'SP 24': '2/17/24',
 'S1 24': '4/15/24',
 'S2 24': '4/15/24',
 'S3 24': '4/15/24',
 'FA 24': '5/24/24',
 'WI 25': '11/12/25'}

Convert the month/day/year format into a datetime object for consistent formatting

In [4]:
for key in enrollment_starts:
    enrollment_starts[key] = pd.to_datetime(enrollment_starts[key], format='%m/%d/%y')

enrollment_starts

{'FA 22': Timestamp('2022-05-20 00:00:00'),
 'WI 23': Timestamp('2023-11-07 00:00:00'),
 'SP 23': Timestamp('2023-02-18 00:00:00'),
 'FA 23': Timestamp('2023-05-26 00:00:00'),
 'WI 24': Timestamp('2024-11-14 00:00:00'),
 'SP 24': Timestamp('2024-02-17 00:00:00'),
 'S1 24': Timestamp('2024-04-15 00:00:00'),
 'S2 24': Timestamp('2024-04-15 00:00:00'),
 'S3 24': Timestamp('2024-04-15 00:00:00'),
 'FA 24': Timestamp('2024-05-24 00:00:00'),
 'WI 25': Timestamp('2025-11-12 00:00:00')}

Now we can perform the following task.

- The dataset contains datapoints from before the enrollment period opens. This may skew the data, making it appear as if open slots are available for longer than they actually are.
- We will remove these datapoints.

In [5]:
df = df[df['quarter'].map(enrollment_starts) <= df['time']]
df

,time,enrolled,available,waitlisted,total,course,quarter
4,2022-05-20 00:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22
5,2022-05-20 12:00:00,1.0,67.0,0.0,68.0,AAS 10,FA 22
6,2022-05-21 00:00:00,3.0,65.0,0.0,68.0,AAS 10,FA 22
7,2022-05-21 12:00:00,5.0,63.0,0.0,68.0,AAS 10,FA 22
8,2022-05-22 00:00:00,6.0,62.0,0.0,68.0,AAS 10,FA 22
...,...,...,...,...,...,...,...
2772269,2024-11-22 12:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24
2772270,2024-11-23 00:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24
2772271,2024-11-23 12:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24
2772272,2024-11-24 00:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24


We also need columns that measure enrollment priority (senior, junior...) and pass # (1, 2), as these are integral parts of our research question. These can easily be derived with respect to each quarter's enrollment start date.

In [ ]:
# Enrollment start dates for incoming students
fa_fresh_start = {
    'FA 22' : pd.to_datetime("8-17-2022"),
    'FA 23' : pd.to_datetime("8-28-2023"),
    'FA 24' : pd.to_datetime("8-12-2024")
}

def registration_priority(date, quarter):

    # Ensure consistent formatting 
    enrollment_start_date = pd.to_datetime(enrollment_starts[quarter])
    date = pd.to_datetime(date)
    days = int((date - enrollment_start_date).days) # number of days since enrollment has been open
    
    pass_num = 0   
    priority = 0 

    # first week of registration is first pass                               
    if 0 <= days and days < 7:
        pass_num = 1            
        if days == 0:
            priority = 1    # first day of each pass is senior enrollment (1)
        elif days == 1:
            priority = 2    # second day = junior (2)
        elif days == 2: 
            priority = 3    # soph (3)
        else:
            priority = 4    # fresh (4)

    # second week is second pass
    elif 7 <= days and days < 14:
        pass_num = 2 
        if days == 7:
            priority = 1
        elif days == 8:
            priority = 2
        elif days == 9:
            priority = 3
        else:
            priority = 4 

    # afterwards, registration is open to all
    else:
        # these values are just placeholders to represent that anybody can enroll
        pass_num = 3 
        priority = 6        

    # Incoming freshman enrollment is unique
    if 'FA' in quarter:
        days = int((date - fa_fresh_start[quarter]).days)
        
        if 0 <= days and days < 7:
            pass_num = 1
            priority = 5
        elif 7 <= days and days < 14:
            pass_num = 2
            priority = 5
    
    return pass_num, priority

In [8]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)
    df[['pass', 'priority']] = df.apply(lambda row: registration_priority(row['time'], row['quarter']), axis=1, result_type='expand')
df

,time,enrolled,available,waitlisted,total,course,quarter,pass,priority
4,2022-05-20 00:00:00,0.0,68.0,0.0,68.0,AAS 10,FA 22,1,1
5,2022-05-20 12:00:00,1.0,67.0,0.0,68.0,AAS 10,FA 22,1,1
6,2022-05-21 00:00:00,3.0,65.0,0.0,68.0,AAS 10,FA 22,1,2
7,2022-05-21 12:00:00,5.0,63.0,0.0,68.0,AAS 10,FA 22,1,2
8,2022-05-22 00:00:00,6.0,62.0,0.0,68.0,AAS 10,FA 22,1,3
...,...,...,...,...,...,...,...,...,...
2772269,2024-11-22 12:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24,3,6
2772270,2024-11-23 00:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24,3,6
2772271,2024-11-23 12:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24,3,6
2772272,2024-11-24 00:00:00,8.0,17.0,0.0,25.0,WES 268A,FA 24,3,6


In [9]:
df['priority'].value_counts()

priority
6    1724573
5     128778
4     124283
1      31764
3      31431
2      31411
Name: count, dtype: int64

We can see from the value counts that certain subsets are over-represented. It simply doesn't make sense to have so much data outside of first and second pass, when these are the focus of our question.

1. There is a disproportionate number of days of freshman priority (4). This is because we're considering all days after sophomore enrollment, but before the end of the week, to be freshman enrollment.

2. The same is true for incoming freshman enrollment (priorty 5).

3. The same is true for when enrollment is opened to all students (priority 6).

Realistically, we only need to know how many seats are available during the first day of someone's first pass, first day of their second pass, and first day of open enrollment. Anything beyond this will simply skew our predictive model.

In [24]:
# Find the first date for each group
first_dates = df.groupby(['quarter', 'pass', 'priority'])['time'].min().reset_index()

# Merge with the original df to keep all unique (quarter, pass, priority) entries on the first date
df = pd.merge(df, first_dates, on=['quarter', 'pass', 'priority', 'time'], how='inner')

df

,time,enrolled,available,waitlisted,total,course,quarter,pass,priority
0,2022-05-20,0.0,68.0,0.0,68.0,AAS 10,FA 22,1,1
1,2022-05-21,3.0,65.0,0.0,68.0,AAS 10,FA 22,1,2
2,2022-05-22,6.0,62.0,0.0,68.0,AAS 10,FA 22,1,3
3,2022-05-23,6.0,62.0,0.0,68.0,AAS 10,FA 22,1,4
4,2022-05-27,32.0,36.0,0.0,68.0,AAS 10,FA 22,2,1
...,...,...,...,...,...,...,...,...,...
80959,2024-06-02,1.0,24.0,0.0,25.0,WES 268A,FA 24,2,3
80960,2024-06-03,1.0,24.0,0.0,25.0,WES 268A,FA 24,2,4
80961,2024-06-07,1.0,24.0,0.0,25.0,WES 268A,FA 24,3,6
80962,2024-08-12,1.0,24.0,0.0,25.0,WES 268A,FA 24,1,5


In [22]:
df['priority'].value_counts()

priority
4    15915
3    15890
2    15887
1    15876
5     9178
6     8218
Name: count, dtype: int64

In [23]:
df['pass'].value_counts()

pass
1    37040
2    35706
3     8218
Name: count, dtype: int64

This is much more representative of the analysis we want to do, which prioritizes senior to freshman registration during first and second pass. 

In [25]:
df.to_csv('new_enrollment_data.csv', index=False)